In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


### Install Gradio

In [ ]:
!pip install gradio

In [ ]:
# Import necessary libraries
import torch
from torchvision import models
from albumentations.pytorch import ToTensor
import gradio as gr
import torch.nn as nn
import numpy as np

### Load the existing model

In [ ]:
# Specify a path
PATH = "/content/drive/MyDrive/Individual_Project/VGG_pretrained/VGG_pretrained.pt"
# Load
model_loaded = torch.load(PATH)
model_loaded.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
# test if the model works
x = torch.randn([3,224,224])
model_loaded(x.unsqueeze(0).cuda())

tensor([[   0.0000, -350.2867, -117.7827]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward>)

In [ ]:
def deploy(img):
    # img shape = 224,224,3
    transform1=ToTensor()
    j=transform1(image=img)
    y=model_loaded(j["image"].unsqueeze(0).cuda())
    lbl= int(y.max(1)[1])

    if lbl==0:
        return "Cell belongs to class A"
    elif lbl==1:
        return "Cell belongs to class B" 
    else:
        return "Cell Belongs to class C" 

In [ ]:
iface = gr.Interface(deploy,
             gr.inputs.Image(shape=(224, 224)),
             gr.outputs.Label(num_top_classes=3)) #click on the link below to open in browser
iface.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://35277.gradio.app
Interface loading below...


KeyboardInterrupt: ignored

In [ ]:
# Here is important point
# from iface it will load image as 224,224,3
# but our model requires the shape 3,224,224
x = np.zeros((224, 224, 3))
print(x.shape)
# Here it is very interesting that ToTensor will transform the image shape from 224,224,3 to 3,224,224 but why?
transform1=ToTensor()
j=transform1(image=x)

# Learn Tensor and transform
print(j['image'].shape)

(224, 224, 3)
torch.Size([3, 224, 224])


In [ ]:
j["image"].unsqueeze(0).shape

torch.Size([1, 3, 224, 224])

In [ ]:
y=model(j["image"].unsqueeze(0))
y

tensor([[-1.0956, -1.0957, -1.1046]], grad_fn=<LogSoftmaxBackward>)

In [ ]:
int(y.max(1)[1])

0